<a href="https://colab.research.google.com/github/HoangHungLN/MachineLearning_Assignment/blob/main/Extended_Assignment/notebooks/Extended_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
import json
import os
import getpass, os, subprocess, textwrap


os.makedirs("data", exist_ok = True)
!wget https://raw.githubusercontent.com/HoangHungLN/MachineLearning_Assignment/refs/heads/main/Extended_Assignment/data/train.json -O data/train.json
!wget https://raw.githubusercontent.com/HoangHungLN/MachineLearning_Assignment/refs/heads/main/Extended_Assignment/data/dev.json -O data/dev.json

DATA_FILE = 'data/train.json'
DEV_FILE = 'data/dev.json'
PARAM_DIR = 'Extended_Assigment/parameters'

--2025-11-10 05:57:35--  https://raw.githubusercontent.com/HoangHungLN/MachineLearning_Assignment/refs/heads/main/Extended_Assignment/data/train.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27164412 (26M) [text/plain]
Saving to: ‘data/train.json’

data/train.json     100%[===================>]  25.91M  --.-KB/s    in 0.1s    

2025-11-10 05:57:35 (228 MB/s) - ‘data/train.json’ saved [27164412/27164412]

--2025-11-10 05:57:36--  https://raw.githubusercontent.com/HoangHungLN/MachineLearning_Assignment/refs/heads/main/Extended_Assignment/data/dev.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133

In [47]:
with open(DATA_FILE, 'r', encoding='utf-8') as f:
    raw_train = json.load(f)
with open(DEV_FILE, 'r', encoding='utf-8') as f:
    raw_dev = json.load(f)

TAG_KEY = "labels"

X_train = [ex['sentence'] for ex in raw_train]
Y_train = [ex[TAG_KEY]     for ex in raw_train]
X_dev   = [ex['sentence'] for ex in raw_dev]
Y_dev   = [ex[TAG_KEY]     for ex in raw_dev]

TAG_KEY = labels
38218 câu train | 5527 câu dev


In [48]:
UNK = "<UNK>"
min_freq = 1

word_freq = Counter(w for sent in X_train for w in sent)
vocab = {w for w,c in word_freq.items() if c > min_freq}
vocab.add(UNK)
vocab = sorted(vocab)

tag_set = sorted({t for tags in Y_train for t in tags})

word2id = {w:i for i,w in enumerate(vocab)}
tag2id  = {t:i for i,t in enumerate(tag_set)}
id2tag  = {i:t for t,i in tag2id.items()}

def map_unk(sent):
    return [w if w in word2id else UNK for w in sent]

X_train = [map_unk(s) for s in X_train]
X_dev   = [map_unk(s) for s in X_dev]

(43194,
 45,
 ['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the'],
 ['NNP', 'NNP', ',', 'CD', 'NNS', 'JJ', ',', 'MD', 'VB', 'DT'])

In [49]:
import numpy as np

def estimate_hmm_supervised(X, Y, tag2id, word2id,
                            alpha_pi=1.0, alpha_A=1.0, alpha_B=1e-3):
    K, V = len(tag2id), len(word2id)
    pi_cnt = np.full(K,     alpha_pi, dtype=np.float64)
    A_cnt  = np.full((K,K), alpha_A,  dtype=np.float64)
    B_cnt  = np.full((K,V), alpha_B,  dtype=np.float64)

    for words, tags in zip(X, Y):
        if not words:
            continue
        pi_cnt[tag2id[tags[0]]] += 1
        for t in range(len(words)):
            j = tag2id[tags[t]]
            w = word2id[words[t]]
            B_cnt[j, w] += 1
            if t < len(words)-1:
                i = tag2id[tags[t]]
                k = tag2id[tags[t+1]]
                A_cnt[i, k] += 1

    pi = pi_cnt / pi_cnt.sum()
    A  = A_cnt / A_cnt.sum(axis=1, keepdims=True)
    B  = B_cnt / B_cnt.sum(axis=1, keepdims=True)
    return pi, A, B

pi, A, B = estimate_hmm_supervised(X_train, Y_train, tag2id, word2id)

sum(pi) = 1.0000000000000002
row sums A (5 mẫu) = [1.0, 1.0, 1.0, 1.0, 1.0]
row sums B (5 mẫu) = [1.0, 1.0, 1.0, 1.0, 1.0]


In [50]:
os.makedirs(PARAM_DIR, exist_ok=True)

np.savez_compressed(
    os.path.join(PARAM_DIR, "hmm_params_supervised.npz"),
    pi=pi, A=A, B=B
)

config = {
    "UNK": UNK,
    "tag_key": TAG_KEY,
    "vocab":   vocab,      # giữ nguyên thứ tự tương ứng với ma trận B
    "tag_set": tag_set,    # giữ nguyên thứ tự tương ứng với các ma trận
    "alpha": {"pi":1.0, "A":1.0, "B":1e-3}
}
with open(os.path.join(PARAM_DIR, "tag_word_maps.json"), "w", encoding="utf-8") as f:
    json.dump(config, f, ensure_ascii=False, indent=2)

print("Saved files:", os.listdir(PARAM_DIR))

Saved files: ['hmm_params_supervised.npz', 'tag_word_maps.json']


In [ ]:
from modules.forward_algorithm import *

print("\n--- Đang chạy thử Giải thuật Forward ---")

# Chúng ta KHÔNG CẦN TẢI LẠI (load) các file đã lưu,
# vì các biến `pi`, `A`, `B`, `X_dev`, `word2id` đã có sẵn trong bộ nhớ.

# 2.1. Lấy một câu từ tập dev để làm chuỗi quan sát (O)
# (Lưu ý: `X_dev` đã được map_unk ở trên)
test_sentence_words = X_dev[1]  # Lấy câu thứ 2 từ tập dev
print(f"Câu kiểm tra: {' '.join(test_sentence_words)}")

# 2.2. Chuyển câu (words) sang chuỗi quan sát O (indices)
O_indices = []
idx_unk = word2id[UNK] # Lấy chỉ số của token UNK
for word in test_sentence_words:
    # Lấy chỉ số, nếu không có thì dùng chỉ số UNK
    idx = word2id.get(word, idx_unk)
    O_indices.append(idx)
O_indices = np.array(O_indices)

print(f"Chuỗi quan sát (O) dạng chỉ số: {O_indices[:15]}...")

# 2.3. Gọi hàm forward
if len(O_indices) > 0:
    total_prob = forward_algorithm(O_indices, pi, A, B)
    print("\n--- Kết quả Forward ---")
    print(f"Tổng xác suất P(O|lambda): {total_prob}")
else:
    print("Câu kiểm tra bị rỗng, không thể chạy Forward.")